# Analisando dados dos resultados da eleição de 2016 e 2018

In [1]:
import pandas as pd

In [2]:
zonas = pd.read_csv('dados/bairros_zonas.csv')
zonas.head()

,Zona,Bairro
0,4,BOTAFOGO
1,4,HUMAITA
2,4,FLAMENGO
3,4,URCA
4,5,COPACABA\rNA


In [3]:
resultados_2016 = pd.read_csv('dados/votacao_candidato_munzona_2016_RJ.csv', delimiter=';', encoding = "ISO-8859-1")
resultados_2016.head(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,36,PTC,Partido Trabalhista Cristão,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
1,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
2,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0


In [4]:
resultados_2018 = pd.read_csv('dados/votacao_candidato_munzona_2018_RJ.csv', delimiter=';', encoding = "ISO-8859-1")
resultados_2018.head(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,16/11/2019,18:45:32,2018,2,Eleição Ordinária,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,45,PSDB,Partido da Social Democracia Brasileira,190000050094,O RIO TEM JEITO,SOLIDARIEDADE / PSDB / PPS,5,SUPLENTE,N,22
1,16/11/2019,18:45:32,2018,2,Eleição Ordinária,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,31,PHS,Partido Humanista da Solidariedade,190000050173,PARTIDO ISOLADO,PHS,3,ELEITO POR MÉDIA,N,2
2,16/11/2019,18:45:32,2018,2,Eleição Ordinária,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,12,PDT,Partido Democrático Trabalhista,190000050590,PARTIDO ISOLADO,PDT,5,SUPLENTE,N,12


**Definindo funções**

In [5]:
def filtrar_eleitos_municipio(df, municipio):
    
    return df.loc[(df['NM_MUNICIPIO'] == municipio) & 
               ((df['DS_SIT_TOT_TURNO'] == 'ELEITO') |
               (df['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') |
               (df['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP'))][['ANO_ELEICAO', 'NR_TURNO', 
              'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO', 
              'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_SITUACAO_CANDIDATURA', 'SG_PARTIDO', 
              'NM_PARTIDO', 'QT_VOTOS_NOMINAIS', 'ST_VOTO_EM_TRANSITO', 
              'DS_SIT_TOT_TURNO']]

In [6]:
def get_total_votos(df):
    
    total_votos = df.groupby('NM_CANDIDATO').agg({'QT_VOTOS_NOMINAIS': 'sum'}).reset_index()
    total_votos = total_votos.rename(columns={"QT_VOTOS_NOMINAIS": "TOTAL_VOTOS"})
    
    return total_votos

In [7]:
def add_total_votos(df):
    
    total_votos = get_total_votos(df)
    df = df.merge(total_votos, on='NM_CANDIDATO')
    
    return df

In [8]:
def add_porcentagem_votos(df):
    
    df['PROPORCAO'] = df['QT_VOTOS_NOMINAIS']/df['TOTAL_VOTOS']
    df['PORCENTAGEM'] = df['PROPORCAO']*100
    
    return df

In [9]:
def filtrar_top_votos(df, indice):
    return df.sort_values(['NM_CANDIDATO', 'QT_VOTOS_NOMINAIS'], 
                          ascending=False).groupby('NM_CANDIDATO').head(indice)

In [10]:
def gerar_top_votacoes_candidato(df, indice, municipio):
    
    df = filtrar_eleitos_municipio(df, municipio)
    df = add_total_votos(df)
    df = filtrar_top_votos(df, indice)
    df = add_porcentagem_votos(df)
    
    return df

## Eleições 2016
### Filtrando top 5 de quantidade de votos por zona eleitoral e candidata/candidato no Rio de Janeiro

In [11]:
resultados_2016.groupby('DS_CARGO').agg({'SQ_CANDIDATO': 'count'})

,SQ_CANDIDATO
DS_CARGO,
Prefeito,2302
Vereador,212500


In [12]:
votacoes_rj_2016 = gerar_top_votacoes_candidato(resultados_2016, 5, 'RIO DE JANEIRO')
votacoes_rj_2016.head(3)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
2427,2016,1,RIO DE JANEIRO,25,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,11081,N,ELEITO POR QP,20678,0.535884,53.588355
2426,2016,1,RIO DE JANEIRO,241,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,1697,N,ELEITO POR QP,20678,0.082068,8.206790
2510,2016,1,RIO DE JANEIRO,243,13,Vereador,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,1184,N,ELEITO POR QP,20678,0.057259,5.725892


**Testando resultados**

In [14]:
votacoes_rj_2016.loc[votacoes_rj_2016['NM_URNA_CANDIDATO'] == 'MARIELLE FRANCO'][['NR_ZONA',
                                                                        'NM_CANDIDATO', 
                                                                        'QT_VOTOS_NOMINAIS',
                                                                        'TOTAL_VOTOS', 
                                                                        'PROPORCAO',
                                                                        'PORCENTAGEM'
                                                                       ]]

,NR_ZONA,NM_CANDIDATO,QT_VOTOS_NOMINAIS,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
4464,16,MARIELLE FRANCISCO DA SILVA,2237,46502,0.048105,4.810546
4487,4,MARIELLE FRANCISCO DA SILVA,1926,46502,0.041418,4.141757
4501,7,MARIELLE FRANCISCO DA SILVA,1837,46502,0.039504,3.950368
4481,161,MARIELLE FRANCISCO DA SILVA,1688,46502,0.036300,3.629951
4514,9,MARIELLE FRANCISCO DA SILVA,1333,46502,0.028665,2.866543


In [15]:
zonas.loc[(zonas['Zona'] == 16) | 
          (zonas['Zona'] == 4) | 
          (zonas['Zona'] == 7) | 
          (zonas['Zona'] == 161) | 
          (zonas['Zona'] == 9)]

,Zona,Bairro
0,4,BOTAFOGO
1,4,HUMAITA
2,4,FLAMENGO
3,4,URCA
6,7,TIJUCA
15,9,BARRA DA\rTIJUCA
16,9,CAMORIM
17,9,VARGEM\rGRANDE
18,9,VARGEM\rPEQUENA
19,9,RECREIO


Resultados estão de acordo com a notícias publicada pela Folha de [São Paulo](https://www1.folha.uol.com.br/cotidiano/2018/03/marielle-franco-foi-mais-votada-em-bairros-nobres-do-rio-de-janeiro.shtml)

In [17]:
votacoes_rj_2016.to_csv('dados/top_5_votacao_2016_rj.csv', index=False)

## Eleições 2018

### Filtrando top 5 de quantidade de votos por zona eleitoral e candidata/candidato no Rio de Janeiro

In [18]:
resultados_2018.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 'ST_VOTO_EM_TRANSITO',
       'QT_VOTOS_NOMINAIS'],
      dtype='object')

In [19]:
resultados_2018.groupby('DS_CARGO').agg({'SQ_CANDIDATO': 'count'})

,SQ_CANDIDATO
DS_CARGO,
Deputado Estadual,432612
Deputado Federal,202947
Governador,2196
Senador,2745


In [20]:
votacoes_rj_2018 = gerar_top_votacoes_candidato(resultados_2018, 5, 'RIO DE JANEIRO')
votacoes_rj_2018.head(3)

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
640,2018,1,RIO DE JANEIRO,22,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,21,N,ELEITO POR QP,279,0.075269,7.526882
637,2018,1,RIO DE JANEIRO,16,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,16,N,ELEITO POR QP,279,0.057348,5.734767
669,2018,1,RIO DE JANEIRO,4,6,Deputado Federal,190000624672,WLADIMIR BARROS ASSED MATHEUS DE OLIVEIRA,WLADIMIR GAROTINHO,APTO,PRP,Partido Republicano Progressista,14,N,ELEITO POR QP,279,0.050179,5.017921


In [21]:
votacoes_rj_2018.to_csv('dados/top_5_votacao_2018_rj.csv', index=False)